In [112]:
import numpy as np
import pandas as pd

In [113]:
def normalise(x, mean, std):
    return (x - mean) / std

In [114]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [115]:
def cost(x, y, theta, lamda):
    predict = sigmoid(x @ theta)
    error = -y * np.log(predict) - (1 - y) * np.log(1 - predict)
    temp = theta
    temp[0] = 0
    temp = temp * temp
    extra = temp.sum() * lamda / 2
    return (error.sum() + extra) / len(y)

In [116]:
def cost_gradient(x, y, theta, alpha, lamda):
    predictions = alpha * (x.T @ (sigmoid(x @ theta) - y)) / len(y)
    temp = theta
    temp[0] = 0
    temp = temp * lamda / len(y)
    return predictions + temp

In [117]:
def train(x, y, theta, epochs, alpha, lamda):
    for i in range(epochs):
        theta -= cost_gradient(x, y, theta, alpha, lamda)
#         print("epoch " + str(i+1) + ": ", cost(x, y, theta, lamda))
    return theta

In [118]:
def predict(x, theta):
    return sigmoid(x @ theta)

In [119]:
df = pd.read_csv('diabetes.csv', header = None)
df.shape

(768, 9)

In [120]:
x = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
for i in range(x.shape[1]):
    me = x.T[i].mean()
    std = np.std(x.T[i])
    for j in range(x.shape[0]):
        x[j][i] = normalise(x[j][i], me, std)
X = np.ones(shape = (x.shape[0], x.shape[1] + 1))
X[:, 1:] = x
Y = Y.reshape((768, 1))

In [121]:
x_train = X[:600]
y_train = Y[:600]
x_test = X[600:]
y_test = Y[600:]

In [122]:
epochs = 1000
l_rate = 0.001
theta = np.zeros(shape = (X.shape[1], 1))

In [123]:
 theta = train(x_train, y_train, theta, epochs, l_rate, 0.)

In [124]:
theta

array([[-1.51997986e-04],
       [ 8.04900390e-02],
       [ 1.84713450e-01],
       [ 1.75227261e-02],
       [ 2.20427181e-02],
       [ 4.89082937e-02],
       [ 1.27008320e-01],
       [ 6.69979046e-02],
       [ 7.62131552e-02]])

In [125]:
y_predict = predict(x_test, theta)
y_predict, cost(x_train, y_train, theta, 0.)

(array([[0.42191117],
        [0.39123127],
        [0.44469644],
        [0.62568767],
        [0.53621027],
        [0.47852032],
        [0.6508632 ],
        [0.37643491],
        [0.56918614],
        [0.41056759],
        [0.4501105 ],
        [0.5948713 ],
        [0.66917523],
        [0.4972929 ],
        [0.62272366],
        [0.4109577 ],
        [0.46334534],
        [0.33269801],
        [0.56603554],
        [0.41381771],
        [0.5017408 ],
        [0.46227616],
        [0.68762247],
        [0.46456171],
        [0.41391111],
        [0.47750772],
        [0.40556176],
        [0.45738925],
        [0.51700852],
        [0.3893661 ],
        [0.48800712],
        [0.4430441 ],
        [0.41148036],
        [0.45165436],
        [0.43457154],
        [0.51950076],
        [0.46298325],
        [0.44034747],
        [0.54494755],
        [0.37487326],
        [0.44549245],
        [0.48157188],
        [0.53964037],
        [0.41316947],
        [0.46593278],
        [0

In [129]:
cnt = 0
y_predict_new = np.where(y_predict >= 0.55, 1, 0)
for i in range(168):
    if y_predict_new[i] == y_test[i]:
        cnt += 1
print(cnt * 100 / 168) # Accuracy

77.38095238095238


# Regularisation

In [101]:
lamdas = np.arange(0, 1, 0.01)
lamdas

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])

In [102]:
for lamda in lamdas:
    theta = train(x_train, y_train, theta, epochs, l_rate, lamda)
    print("lamda " + str(lamda) + ": " + str(cost(x_train, y_train, theta, lamda)))
# Thus lamda 0.02 is best

lamda 0.0: 0.540639132001548
lamda 0.01: 0.53954811101587
lamda 0.02: 0.5391541728069034
lamda 0.03: 0.5394376243258335
lamda 0.04: 0.5403790780669232
lamda 0.05: 0.5419420154800537
lamda 0.06: 0.54406446327486
lamda 0.07: 0.5466589923669318
lamda 0.08: 0.5496192277698773
lamda 0.09: 0.5528302422474628
lamda 0.1: 0.5561798905314461
lamda 0.11: 0.5595684976780094
lamda 0.12: 0.5629152489711481
lamda 0.13: 0.5661607915982944
lamda 0.14: 0.5692665447681613
lamda 0.15: 0.5722117725246982
lamda 0.16: 0.5749895786975888
lamda 0.17: 0.5776027752972649
lamda 0.18: 0.5800602377464641
lamda 0.19: 0.58237403460194
lamda 0.2: 0.5845573780636644
lamda 0.21: 0.5866232992061605
lamda 0.22: 0.5885838910918074
lamda 0.23: 0.5904499561103559
lamda 0.24: 0.59223091579794
lamda 0.25: 0.5939348739549214
lamda 0.26: 0.5955687563705436
lamda 0.27: 0.5971384777462215
lamda 0.28: 0.598649106939876
lamda 0.29: 0.6001050158139991
lamda 0.3: 0.6015100059636483
lamda 0.31: 0.6028674128042273
lamda 0.32: 0.60418018

In [111]:
theta = train(x_train, y_train, theta, epochs, l_rate, 0.02)
y_predict = predict(x_test, theta)
cnt = 0
y_predict_new = np.where(y_predict >= 0.55, 1, 0)
for i in range(168):
    if y_predict_new[i] == y_test[i]:
        cnt += 1
print(cnt * 100 / 168) # Accuracy

78.57142857142857
